## 1. Getting VizDoom up and running

In [2]:
# import VizDoom for game env
from vizdoom import *
# Import random for action sampling
import random
# Import time for sleeping
import time
# import numpy for identity matrix
import numpy as np

from matplotlib import pyplot as plt

## 2. Converting it to a Gym Environment

In [3]:
# Import environment base class from OpenAI Gym
from gymnasium import Env
# Import gym spaces
from gymnasium.spaces import Discrete, Box
# Import Opencv for greyscaling observations
import cv2

# Import environment checker
from stable_baselines3.common import env_checker
# Discrete(3).sample() returns a number from 0, 1, 2 -> used as index to select action
# Box(low=0, high=10, shape=(10,10)).sample() -> getting 10x10 array with low=0 and high=10

In [4]:
# Create VizDoom OpenAI Gym Environment
class VizDoomGym(Env): 
    def __init__(self, render=False):
        """
        Function called when we start the env.
        """

        # Inherit from Env
        super().__init__()
        
        # Set up game
        self.game = DoomGame()
        self.game.load_config('VizDoom/scenarios/basic.cfg')
        

        # Whether we want to render the game 
        if render == False:
            self.game.set_window_visible(False)
        else:
            self.game.set_window_visible(True)

        # Start the game
        self.game.init()
        
        # Create action space and observation space
        self.observation_space = Box(low=0, high=255, shape=(100, 160, 1), dtype=np.uint8)
        self.action_space = Discrete(3)

    
    def step(self, action):
        """
        How we take a step in the environment.
        """

        # Specify action and take step
        actions = np.identity(3, dtype=np.uint8)
        reward = self.game.make_action(actions[action], 4) # get action using index -> left, right, shoot
        
        # Get all the other stuff we need to return 
        if self.game.get_state():  # if nothing is
            state = self.game.get_state().screen_buffer
            state = self.grayscale(state)  # Apply Grayscale
            ammo = self.game.get_state().game_variables[0] 
            info = ammo
        # If we dont have anything turned from game.get_state
        else:
            # Return a numpy zero array
            state = np.zeros(self.observation_space.shape)
            # Return info (game variables) as zero
            info = 0

        info = {"info":info}
        done = self.game.is_episode_finished()
        truncated = False  # Assuming it's not truncated, modify if applicable
        
        return state, reward, done, truncated, info

    
    def render(self):
        """
        Define how to render the game environment.
        """
        pass

    
    def reset(self, seed=None):
        """
        Function for defining what happens when we start a new game.
        """
        if seed is not None:
            self.game.set_seed(seed)
            
        self.game.new_episode()
        state = self.game.get_state().screen_buffer  # Apply Grayscale

        return self.grayscale(state), {}

    
    def grayscale(self, observation):
        """
        Function to grayscale the game frame and resize it.
        observation: gameframe
        """
        # Change colour channels 
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)

        # Reduce image pixel size for faster training
        resize = cv2.resize(gray, (160,100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize,(100, 160,1))
        return state

    def close(self):
        """
        Call to close down the game.
        """
        self.game.close()


In [5]:
env = VizDoomGym(render=True)

Environment checker

In [6]:
env_checker.check_env(env)

In [7]:
env.close()

## Hyperparameter Tune

In [8]:
# Importing the optimzation frame - HPO
import optuna
# PPO algo for RL
from stable_baselines3 import PPO
# Bring in the eval policy method for metric calculation
from stable_baselines3.common.evaluation import evaluate_policy
# Import the sb3 monitor for logging 
from stable_baselines3.common.monitor import Monitor
# Import the vec wrappers to vectorize and frame stack
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
# Import os to deal with filepaths
import os

/Users/yeungivan/anaconda3/envs/Doom/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
LOG_DIR = './logs/log_basic/tuning'
OPT_DIR = './opt/'

In [10]:
# Function to return test hyperparameters - define the object function
def optimize_ppo(trial): 
    return {
        'n_steps':trial.suggest_int('n_steps', 2048, 8192),
        'learning_rate':trial.suggest_float('learning_rate', 1e-5, 1e-4),
    }

In [11]:
SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(1))

In [12]:
# Run a training loop and return mean reward 
def optimize_agent(trial):
    try:
        model_params = optimize_ppo(trial) 

        # Create environment 
        env = VizDoomGym()
        env = Monitor(env, LOG_DIR)
        env = DummyVecEnv([lambda: env])
        env = VecFrameStack(env, 4, channels_order='last')

        # Create algo 
        model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=0, **model_params)
        model.learn(total_timesteps=30000)
        #model.learn(total_timesteps=100000)

        # Evaluate model 
        mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5)
        env.close()

        SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(trial.number))
        model.save(SAVE_PATH)

        return mean_reward

    except Exception as e:
        return -1000

In [13]:
# Creating the experiment 
study = optuna.create_study(direction='maximize')
study.optimize(optimize_agent, n_trials=5, n_jobs=1)
#study.optimize(optimize_agent, n_trials=100, n_jobs=1)

[I 2024-04-08 17:41:45,738] A new study created in memory with name: no-name-153ffc2b-3c95-4455-868e-1780a06ff363


: 

In [10]:
study.best_params

{'n_steps': 6928,
 'gamma': 0.8296119993363261,
 'learning_rate': 2.296979511732029e-05,
 'clip_range': 0.3688634008030219,
 'gae_lambda': 0.8002547385297311}

In [11]:
study.best_trial

FrozenTrial(number=0, state=TrialState.COMPLETE, values=[-1000.0], datetime_start=datetime.datetime(2024, 4, 8, 17, 30, 25, 419385), datetime_complete=datetime.datetime(2024, 4, 8, 17, 30, 25, 420239), params={'n_steps': 6928, 'gamma': 0.8296119993363261, 'learning_rate': 2.296979511732029e-05, 'clip_range': 0.3688634008030219, 'gae_lambda': 0.8002547385297311}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_steps': IntDistribution(high=8192, log=False, low=2048, step=1), 'gamma': FloatDistribution(high=0.9999, log=False, low=0.8, step=None), 'learning_rate': FloatDistribution(high=0.0001, log=False, low=1e-05, step=None), 'clip_range': FloatDistribution(high=0.4, log=False, low=0.1, step=None), 'gae_lambda': FloatDistribution(high=0.99, log=False, low=0.8, step=None)}, trial_id=0, value=None)

## 3. Setup Callback
Save model at different state of training

In [12]:
# Import os for file nav
import os
# Import callback class from sb3
from stable_baselines3.common.callbacks import BaseCallback

In [13]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
        return True
                                      

In [14]:
CHECKPOINT_DIR = './train/train_basic'
LOG_DIR = './logs/log_basic'

In [15]:
# Create instance of callback
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR) 
# after every 10000 steps of training the model, weights are saved for the pytorch agent

## 5. Train Model


In [16]:
model_params = study.best_params
model_params['n_steps'] = 7488  # set n_steps to 7488 or a factor of 64
# model_params['learning_rate'] = 5e-7
model_params

{'n_steps': 7488,
 'gamma': 0.8296119993363261,
 'learning_rate': 2.296979511732029e-05,
 'clip_range': 0.3688634008030219,
 'gae_lambda': 0.8002547385297311}

In [17]:
# Import PPO for training
from stable_baselines3 import PPO

In [25]:
# Non rendered environment
env = VizDoomGym()

In [26]:
# n_steps: How many steps/frames the agent is going to take and store in the buffer 
# before run through training of actor and critique
# ideally not too close to end of game (300) but somewhere close
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, **model_params)

: 

In [ ]:
model.learn(total_timesteps=100000, callback=callback)

Logging to ./logs/log_basic/PPO_2
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 16.1     |
|    ep_rew_mean     | 19.6     |
| time/              |          |
|    fps             | 28       |
|    iterations      | 1        |
|    time_elapsed    | 72       |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 11.1       |
|    ep_rew_mean          | 48         |
| time/                   |            |
|    fps                  | 21         |
|    iterations           | 2          |
|    time_elapsed         | 193        |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.03136115 |
|    clip_fraction        | 0.315      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.795     |
|    explained_variance   | 0.663      |
|    learni

KeyboardInterrupt: 

## 5. Test Model


In [ ]:
# Import eval policy to test agent
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
# Reload model from disc
model = PPO.load('./train/train_basic/best_model_110000')

In [ ]:
# Create rendered envrironment
env = VizDoomGym(render=True)

In [ ]:
# Evaluate mean reward for 10 games
mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=10)

In [ ]:
mean_reward

85.4

In [ ]:
total_reward = 0
for episode in range(5):
    obs = env.reset()[0]
    done = False
    while not done:
        action, _ = model.predict(obs) # Use model to predict what action to take
        obs, reward, done, _, info = env.step(action) # take the predicted action
        time.sleep(0.25)
        total_reward += reward
    print('Total Reward for episode {} is {}'.format(total_reward, episode))
    time.sleep(2)
          
    

Total Reward for episode 91.0 is 0
Total Reward for episode 170.0 is 1
Total Reward for episode 249.0 is 2
Total Reward for episode 340.0 is 3
Total Reward for episode 407.0 is 4
